In [68]:
module Ev3dev

import Base: read, parse, write, call

parse(::Type{ASCIIString}, s::ASCIIString) = s
parse(::Type{Vector{ASCIIString}}, s::ASCIIString) = split(s)

immutable Brick
    root_path::AbstractString
    
    Brick(root_path::AbstractString="/") = new(root_path)
end

immutable Attribute{T, Read, Write}
    name::Symbol
    stream::IOStream
end

function call{T, Read, Write}(::Type{Attribute{T, Read, Write}}, brick::Brick, relative_path::AbstractString, name)
    path = joinpath(brick.root_path, relative_path, string(name))
    isfile(path) || error("file not found: $path")
    Read && (isreadable(path) || error("read access for attribute $(path) was requested, but file is not readable"))
    Write && (iswritable(path) || error("write access for attribute $(path) was requested, but file is not writable"))
    stream = open(path, Read, Write, false, Write, false)
    Attribute{T, Read, Write}(name, stream)
end

typealias ReadOnly{T} Attribute{T, true, false}
typealias WriteOnly{T} Attribute{T, false, true}
typealias ReadWrite{T} Attribute{T, true, true}
    
call{T, X}(attr::Attribute{T, true, X}) = parse(T, chomp(strip(readline(seekstart(attr.stream)), '\0')))
function call{T, X}(attr::Attribute{T, X, true}, value::T)
    write(seekstart(attr.stream), string(value))
    flush(attr.stream)
end

abstract Class{Name}

function call{T <: Class}(::Type{T}, brick::Brick, path::AbstractString)
    attr = T()
    for name in fieldnames(T)
        field_type = fieldtype(T, name)
        setfield!(attr, name, field_type(brick, path, name))
    end
    attr
end

abstract Driver{Name}
type Device{C <: Class, D <: Driver}
    attr::C
    commands::Set{ASCIIString}
    
    function Device(brick, path)
        attr = C(brick, path)
        
        commands = Set{ASCIIString}(attr.commands())
        new(attr, commands)
    end
end

name{C <: Class}(::Type{C}) = C.super.parameters[1]
class_path{C <: Class, D <: Driver}(::Type{Device{C, D}}) = joinpath("sys/class", replace(string(name(C)), '_', '-'))
name{N}(::Type{Driver{N}}) = N
driver_name{C, D <: Driver}(::Type{Device{C, D}}) = replace(string(name(D)), '_', '-')

type TachoMotor <: Class{:tacho_motor}
    address::ReadOnly{ASCIIString}
    commands::ReadOnly{Vector{ASCIIString}}
    driver_name::ReadOnly{ASCIIString}
    command::WriteOnly{ASCIIString}
    count_per_rot::ReadOnly{Int}
    duty_cycle::ReadOnly{Int}
    duty_cycle_sp::ReadWrite{Int}    
    speed_sp::ReadWrite{Int}
    position_sp::ReadWrite{Int}
    polarity::ReadWrite{ASCIIString}
    stop_action::ReadWrite{ASCIIString}
    stop_actions::ReadOnly{Vector{ASCIIString}}
    
    TachoMotor() = new()
end

typealias LargeMotor Device{TachoMotor, Driver{:lego_ev3_l_motor}}
typealias MediumMotor Device{TachoMotor, Driver{:lego_ev3_m_motor}}

type LegoSensor <: Class{:lego_sensor}
    address::ReadOnly{ASCIIString}
    commands::ReadOnly{Vector{ASCIIString}}
    driver_name::ReadOnly{ASCIIString}
    command::WriteOnly{ASCIIString}
    value0::ReadOnly{Int}
    value1::ReadOnly{Int}
    value2::ReadOnly{Int}
    value3::ReadOnly{Int}
    value4::ReadOnly{Int}
    value5::ReadOnly{Int}
    value6::ReadOnly{Int}
    value7::ReadOnly{Int}
    num_values::ReadOnly{Int}
    decimals::ReadOnly{Int}
    modes::ReadOnly{Vector{ASCIIString}}
    mode::ReadWrite{ASCIIString}
    bin_data::ReadOnly{ASCIIString}
    bin_data_format::ReadOnly{ASCIIString}
    poll_ms::ReadWrite{Int}
    
    LegoSensor() = new()
end

typealias Ultrasound Device{LegoSensor, Driver{:lego_ev3_us}}
typealias ColorSensor Device{LegoSensor, Driver{:lego_ev3_color}}

command(dev::Device, data::AbstractString) = in(data, dev.commands) ? dev.attr.command(data) : error("command $(data) not supported by device")

function values(sensor::Device{LegoSensor})
    vals = Vector{Float64}(sensor.attr.num_values())
    dec = sensor.attr.decimals()
    multiplier = 10.0 ^ (-dec)
    for j = 1:length(vals)
        vals[j] = multiplier * getfield(sensor.attr, symbol(:value, j-1))()
    end
    vals
end

function find_devices{T <: Device}(::Type{T}, brick::Brick)
    path = joinpath(brick.root_path, class_path(T))
    devices = Vector{T}()
    driver = driver_name(T)
    for dir in readdir(path)
        device_driver = open(joinpath(path, dir, "driver_name")) do f
            readchomp(f)
        end
        if device_driver == driver
            push!(devices, T(brick, joinpath(path, dir)))
        end
    end
    devices
end

function find_device_at_address{T <: Device}(::Type{T}, brick::Brick, address::AbstractString)
    for dev in find_devices(T, brick)
        if dev.attr.address() == address
            return dev
        end
    end
    nothing
end

end

import Ev3dev

In [69]:
brick = Ev3dev.Brick("/home/pi/ev3")
motor = Ev3dev.find_devices(Ev3dev.MediumMotor, brick)[1]
ultrasound = Ev3dev.find_devices(Ev3dev.Ultrasound, brick)[1]

Ev3dev.Device{Ev3dev.LegoSensor,Ev3dev.Driver{:lego_ev3_us}}(Ev3dev.LegoSensor(Ev3dev.Attribute{ASCIIString,true,false}(:address,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/address>)),Ev3dev.Attribute{Array{ASCIIString,1},true,false}(:commands,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/commands>)),Ev3dev.Attribute{ASCIIString,true,false}(:driver_name,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/driver_name>)),Ev3dev.Attribute{ASCIIString,false,true}(:command,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/command>)),Ev3dev.Attribute{Int32,true,false}(:value0,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value0>)),Ev3dev.Attribute{Int32,true,false}(:value1,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value1>)),Ev3dev.Attribute{Int32,true,false}(:value2,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value2>)),Ev3dev.Attribute{Int32,true,false}(:value3,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/

In [41]:
ultrasound.attr.value0

Ev3dev.Attribute{Int32,true,false}(:value0,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value0>))

In [49]:
ultrasound.attr.value0()

714

In [50]:
Ev3dev.values(ultrasound)

1-element Array{Float64,1}:
 71.4

In [51]:
motor.attr.address()

"outA"

In [52]:
motor.attr.speed_sp(500)

IOStream(<file /home/pi/ev3/sys/class/tacho-motor/motor0/speed_sp>)

In [53]:
motor.attr.command("run-forever")

IOStream(<file /home/pi/ev3/sys/class/tacho-motor/motor0/command>)

In [54]:
Ev3dev.command(motor, "stop")

IOStream(<file /home/pi/ev3/sys/class/tacho-motor/motor0/command>)